## Mock: 使用Mock來patching方法

Mock 物件的常見用法包含：

1. Patching 方法
2. 記錄在物件上的方法呼叫

Reference:
- [unittest.mock](https://docs.python.org/zh-tw/3/library/unittest.mock-examples.html)

In [15]:
from unittest.mock import MagicMock, Mock, patch, call

### 物件上的方法進行 patch

In [1]:
# 一旦 mock 被呼叫，它的 called 屬性將被設定為 True

from unittest.mock import MagicMock

class ProductionClass:
    def method(self):
        self.something(1, 2, 3)
    def something(self, a, b, c):
        pass

real = ProductionClass()
real.something = MagicMock()
real.method()
real.something.assert_called_once_with(1, 2, 3) # 我們可以使用 assert_called_with() 或 assert_called_once_with() 方法來檢查它是否被使用正確的引數來呼叫

### 將一個物件傳遞給一個方法（或受測系統的某一部分），然後檢查它是否以正確的方式被使用

In [1]:
from unittest.mock import Mock
class ProductionClass:
    def closer(self, something):
        something.close()

real = ProductionClass()
mock = Mock() # 不必做任何額外的事情來為 mock 提供 'close' 方法，存取 close 會建立它
real.closer(mock)
mock.close.assert_called_with()

## Mock類別
- 一個常見的使用案例是在測試的時候 `mock 被程式碼實例化的類別`
- 當你 patch 一個類別時，該類別就會被替換為 mock
- 實例是透過 `呼叫類別建立的`，這代表你可以透過查看 `被 mock 的類別的回傳值來存取「mock 實例」`

In [4]:
from unittest.mock import patch
class Foo:
    
    def __init__(self):
        pass

    def method(self):
        return "foo result"

def some_function():
    # instance = module.Foo() # 某個module下的class
    instance = Foo()
    return instance.method()

# with patch('module.Foo') as mock:
with patch('__main__.Foo') as mock:
    instance = mock.return_value
    instance.method.return_value = 'the result'
    result = some_function()
    assert result == 'the result'

In [5]:
# with patch('module.Foo') as mock:
with patch('__main__.Foo') as mock:
    instance = mock.return_value
    instance.method.return_value = 'the result'
    result = some_function()
    assert result == 'foo result'

AssertionError: 

### 命名你的 mock
- 為你的 mock 命名可能會很有用，這個名稱會顯示在 mock 的 repr 中，且當 mock 出現在測試的失敗訊息中時，名稱會很有幫助

In [8]:
mock = MagicMock(name='foo')
mock

<MagicMock name='foo' id='1799326941264'>

In [10]:
mock.method

<MagicMock name='foo.method' id='1799347054608'>

### 追蹤所有呼叫

In [11]:
# 通常你會想要追蹤對一個方法的多個呼叫
# mock_calls 屬性記錄對 mock 的子屬性以及其子屬性的所有呼叫
mock = MagicMock()
mock.method()

<MagicMock name='mock.method()' id='1799326928208'>

In [12]:
mock.mock_calls

[call.method()]

In [13]:
mock.attribute.method(10, x=53)

<MagicMock name='mock.attribute.method()' id='1799326940752'>

In [14]:
mock.mock_calls

[call.method(), call.attribute.method(10, x=53)]

In [16]:
# 你可以使用 call 物件來建構串列以與 mock_calls 進行比較：
expected = [call.method(), call.attribute.method(10, x=53)]
mock.mock_calls == expected

True

In [17]:
# 回傳 mock 的呼叫的參數不會被記錄
m = Mock()
m.factory(important=True).deliver()

<Mock name='mock.factory().deliver()' id='1799326920848'>

In [18]:
m.mock_calls

[call.factory(important=True), call.factory().deliver()]

In [20]:
# 因為沒有記錄所以會被視為是 True
m.mock_calls[-1] == call.factory(important=False).deliver()

True

### 設定回傳值和屬性

In [21]:
# 在 mock 物件上設定回傳值
mock = Mock()
mock.return_value = 3
mock()

3

In [22]:
# 可以對 mock 上的方法執行相同的操作
mock = Mock()
mock.method.return_value = 3
mock.method()

3

In [23]:
# 回傳值也可以在建構函式中
mock = Mock(return_value=3)
mock()

3

In [24]:
# 在 mock 上進行屬性設置
mock = Mock()
mock.x = 3
mock.x

3